In [88]:
import pandas as pd
from math import sqrt
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from random import randrange
from statistics import mean,stdev
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.decomposition import PCA
from sklearn import model_selection
from sklearn.preprocessing import MinMaxScaler, StandardScaler, Normalizer, Binarizer
from sklearn import linear_model
from sklearn import svm
from sklearn.metrics import mean_squared_error, r2_score

In [89]:
names = ['Id','kitchtmp','kitchhum','livtmp','livhum','lautmp','lauhum','offtmp','offhum','bathtmp','bathhum','outntmp','outnhum','irontmp','ironhum','teentmp','teenhum','partmp','parhum','outtmp','outprsr','outhum','wndspd','visibility','dewpnt','energy']
df = pd.read_csv("D:/Program_files/Python/energy/train.csv",names=names,header=0)
df=df.drop(['Id'],axis=1)
names2 = ['Id','kitchtmp','kitchhum','livtmp','livhum','lautmp','lauhum','offtmp','offhum','bathtmp','bathhum','outntmp','outnhum','irontmp','ironhum','teentmp','teenhum','partmp','parhum','outtmp','outprsr','outhum','wndspd','visibility','dewpnt']
df3 = pd.read_csv("D:/Program_files/Python/energy/test.csv",names=names2,header=0)

In [90]:
def fun2(df,strng):
    q75, q25 = np.percentile(df[strng], [75, 25])
    iqr = q75 - q25

    mn = q25 - iqr*1.5
    mx = q75 + iqr*1.5
    df = df.loc[(df[strng]>mn) & (df[strng]<mx)]
    return df

In [91]:
df = fun2(df,'energy')
df = fun2(df,'bathhum')
df = fun2(df,'outprsr')
df = fun2(df,'wndspd')

In [92]:
y = df['energy'].values
df = df.drop(['energy'],axis=1)
#df = df.drop(['kitchtmp','kitchhum','livtmp','livhum','lautmp','lauhum','offtmp','offhum','bathtmp','outntmp','outnhum','irontmp','ironhum','teentmp','teenhum','partmp','parhum','outtmp','outhum','dewpnt','energy'],axis=1)
X = df.values
X.shape

(12878, 24)

In [93]:
X_train,X_val,y_train,y_val = model_selection.train_test_split(X,y,test_size=0.20,random_state=0)

In [94]:
Id = df3['Id']
df3 = df3.drop(['Id'],axis=1)
X_test = df3.values
X_test.shape

(3945, 24)

In [95]:
#-----------------------------------------------------------------------
#scaler = Normalizer().fit(X)
#normalizedX = scaler.transform(X)
#scaler = MinMaxScaler(feature_range=(0, 1))
#rescaledX = scaler.fit_transform(normalizedX)
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

In [20]:
#if vif is greater than 5 then remove multicollinearity
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(X_train, i) for i in range(X_train.shape[1])]
print(vif)

    VIF Factor
0    20.955238
1    18.432958
2    30.290886
3    24.804533
4    12.057699
5    12.317331
6     9.888768
7    17.771905
8    11.916199
9     1.677975
10   31.868757
11    9.885128
12   18.749210
13   10.456297
14    8.483909
15    7.999832
16   29.146952
17    6.502361
18  151.598511
19    1.409310
20   49.530590
21    1.600941
22    1.044899
23   93.496962


In [96]:
pca = PCA(.99)
pca.fit(X_train)
X_train = pca.transform(X_train)
X_val = pca.transform(X_val)
X_test = pca.transform(X_test)

print (pca.explained_variance_)
print (pca.explained_variance_ratio_)
print (pca.explained_variance_ratio_.cumsum())

[ 9.35077451  7.25237616  1.91808092  1.05162779  0.97911393  0.78192157
  0.52820418  0.47579456  0.36728061  0.26052146  0.17181349  0.14353784
  0.13606696  0.11032965  0.10803068  0.08622303  0.06677926]
[ 0.3896156   0.30218234  0.07992004  0.04381782  0.04079641  0.03258007
  0.02200851  0.01982477  0.01530336  0.01085506  0.0071589   0.00598074
  0.00566946  0.00459707  0.00450128  0.00359263  0.00278247]
[ 0.3896156   0.69179794  0.77171798  0.81553581  0.85633222  0.88891229
  0.91092079  0.93074557  0.94604893  0.95690399  0.96406288  0.97004363
  0.97571308  0.98031015  0.98481143  0.98840406  0.99118652]


In [97]:
reg = svm.SVR(kernel = 'rbf',C=500,gamma=0.1,epsilon=10)
reg.fit(X_train,y_train)

y_predict = reg.predict(X_val)
print(mean_squared_error(y_val,y_predict))

289.43171011143897

In [98]:
#print(reg.coef_)
print(r2_score(y_val,y_predict))
print(reg.score(X_val,y_val))

0.648879159636
0.648879159636


In [99]:
y_test = reg.predict(X_test)

In [100]:
y_results=[]
for i in range(len(y_test)):
    y_results.append(int(y_test[i]))
#    print('%f\t%f\n' % (y_val[i],y_predict[i]))

In [101]:
output = np.column_stack((Id,y_results))
df_results = pd.DataFrame(output.astype('int'),columns=['Observation','Energy'])
df_results.to_csv('D:/Program_files/Python/energy/results.csv',index=False)